In [ ]:
import pandas as pd
import math 
import datetime

import sys
sys.path.append('../src')
import utils_explore as ue

data = ue.all_bets()

In [ ]:
# alltime summary
df_alltime = ue.summary_chart(data,[50,250,1250,5000])
# only last year
df_year19 = ue.summary_chart(data[data['placed-date'] >= datetime.date(2019, 1, 1)], [20, 100, 500, 2500])

In [ ]:
df_alltime[df_alltime.success_rate > 5.0]

In [ ]:
df_year19[df_year19.success_rate > 5.0]